In [ ]:
import pandas as pd # Data Transformation
import requests # establishing connect with the web/api

In [ ]:
url = "https://api.coincap.io/v2/assets"

In [ ]:
response = requests.get(url)
print(response)

In [ ]:
response_data = response.json()
print(response_data)

In [ ]:
print(len(response_data['data']))

In [ ]:
import requests
import pandas as pd
import time

def fetch_all_coins_from_coingecko(pages=40):
    all_coins = []
    per_page = 250
    vs_currency = "usd"

    print("Fetching coin market data from CoinGecko...")

    for page in range(1, pages + 1):
        url = "https://api.coingecko.com/api/v3/coins/markets"
        params = {
            "vs_currency": vs_currency,
            "order": "market_cap_desc",
            "per_page": per_page,
            "page": page,
            "sparkline": "false"
        }

        success = False
        retries = 0

        while not success and retries < 5:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                data = response.json()
                if not data:
                    print(f"No more data at page {page}, stopping.")
                    return pd.DataFrame(all_coins)
                all_coins.extend(data)
                print(f"✅ Fetched page {page}: {len(all_coins)} coins total")
                time.sleep(2)  # Safe delay
                success = True
            elif response.status_code == 429:
                print(f"⚠️ Rate limit hit at page {page}. Waiting 10 seconds...")
                time.sleep(10)
                retries += 1
            else:
                print(f"❌ Failed at page {page}, status code: {response.status_code}")
                return pd.DataFrame(all_coins)

    df = pd.DataFrame(all_coins)
   # df.to_csv("coingecko_all_coins.csv", index=False)
    print(f"\n✅ Done! Total coins fetched: {len(df)}")
    return df

# Run it
df_coins = fetch_all_coins_from_coingecko()

In [ ]:
from google.colab import files  # If using Google Colab

# Save the cleaned dataset
df_coins.to_csv("coingecko_all_coins.csv", index=False) # Changed df to df_coins to access the dataframe

# Download the file (For Google Colab)
files.download("coingecko_all_coins.csv") # Changed the filename to the correct name

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd  # Library for data manipulation and analysis
import numpy as np   # Library for numerical operations
import matplotlib.pyplot as plt  # Library for creating static, animated, and interactive visualizations
import seaborn as sns  # Provides a high-level interface for drawing attractive and informative statistical graphics
import warnings

warnings.filterwarnings('ignore')  # Ignore warnings for better readability

In [ ]:
# Update the path below to the path where your CSV file is stored in Google Drive
crypto_analysis = '/content/drive/MyDrive/PERSONAL_PROJECT/Crypto_Market_Analysis/coingecko_all_coins.csv'
ca_data = pd.read_csv(crypto_analysis)

In [ ]:
# Shape of the dataset
print(ca_data.shape)

In [ ]:
# Display the first 5 rows of the DataFrame to understand what the data looks like
print(ca_data.head())

In [ ]:
# Display the last 5 rows of the DataFrame to understand what the data looks
print(ca_data.tail())

In [ ]:
# Display data information
print(ca_data.info())

In [ ]:
# Display Column Names & Data Types
print(ca_data.dtypes)

In [ ]:
# Display Descriptive Statistics
print(ca_data.describe())

In [ ]:
# Display Â Null Value Count
print(ca_data.isnull().sum())

In [ ]:
# Display Unique Value Count
print(ca_data.nunique())

In [ ]:
# Display Value Counts
print(ca_data.value_counts())

In [ ]:
# Display Correlation Matrix
print(ca_data.select_dtypes(include=np.number).corr())

In [ ]:
# Display Duplicate Check
print(ca_data.duplicated().sum())

In [ ]:
# Structural Cleaning – Drop Unwanted Columns
# 'image' → not needed for analysis
# 'roi' → a nested/dictionary object and mostly null

columns_to_drop = ['roi', 'image']
ca_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')

print("Dropped columns:", columns_to_drop)

In [ ]:
# Handle Missing Values in Critical Fields
# These are essential for reliable analysis
critical_columns = ['id', 'symbol', 'name', 'current_price', 'market_cap']
ca_data.dropna(subset=critical_columns, inplace=True)

print("Removed rows with nulls in critical columns")

In [ ]:
# Fix Data Types – Convert Strings to Datetime
date_columns = ['ath_date', 'atl_date', 'last_updated']

for col in date_columns:
    if col in df.columns:
        ca_data[col] = pd.to_datetime(ca_data[col], errors='coerce')

print("Converted date columns to datetime:", date_columns)

In [ ]:
# Deduplication
before_dedup = ca_data.shape[0]
ca_data.drop_duplicates(subset='id', inplace=True)
after_dedup = ca_data.shape[0]

print(f"Removed {before_dedup - after_dedup} duplicate rows based on 'id'")

In [ ]:
# Handle Outliers or Invalid Values
before_price_filter = ca_data.shape[0]
ca_data = ca_data[ca_data['current_price'] > 0]
after_price_filter = ca_data.shape[0]

print(f"Removed {before_price_filter - after_price_filter} rows with non-positive price")

In [ ]:
# Clean Up Categorical Fields
ca_data['symbol'] = ca_data['symbol'].str.upper()

In [ ]:
# Optional Feature Engineering
ca_data['liquidity_ratio'] = ca_data['total_volume'] / ca_data['market_cap']
ca_data['is_stablecoin'] = ca_data['current_price'].between(0.99, 1.01)  # crude heuristic

In [ ]:
# Final Review and Save

print("\n Final shape:", ca_data.shape)
print("Remaining nulls:\n", ca_data.isnull().sum().sort_values(ascending=False))

# Reset index for cleanliness
ca_data.reset_index(drop=True, inplace=True)

# Save to a new cleaned CSV
cleaned_path = "coingecko_all_coins_cleaned_final.csv"
ca_data.to_csv(cleaned_path, index=False)
print(f"Cleaned dataset saved to: {cleaned_path}")